# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,2.71,76,99,14.02,TF,1730082158
1,1,qaqortoq,60.7167,-46.0333,0.38,99,100,2.29,GL,1730082159
2,2,la ronge,55.1001,-105.2842,4.14,70,100,9.77,CA,1730082160
3,3,benguela,-12.5763,13.4055,23.36,83,54,2.67,AO,1730082161
4,4,tionk essil,12.7856,-16.5217,25.40,81,10,2.52,SN,1730082162


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    alpha = 0.4,
    scale = 0.75,
    color = "City")

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
new_weather_df = city_data_df.loc[(city_data_df["Humidity"]<75) & (city_data_df["Humidity"]>50) & (city_data_df["Max Temp"] < 27) &
(city_data_df["Max Temp"] > 6) & (city_data_df["Wind Speed"] < 6)]

# Drop any rows with null values
new_weather_df = new_weather_df.dropna(how='any')

# Display sample data
new_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,port shepstone,-30.7414,30.4550,17.92,69,0,2.02,ZA,1730082170
13,13,invercargill,-46.4000,168.3500,12.23,71,82,5.36,NZ,1730082173
18,18,shanwei,22.7820,115.3475,25.67,65,33,3.72,CN,1730082178
22,22,bethel,41.3712,-73.4140,6.86,56,100,1.56,US,1730082183
41,41,floresville,29.1336,-98.1561,26.59,62,64,5.61,US,1730082165
...,...,...,...,...,...,...,...,...,...,...
564,564,dunedin,-45.8742,170.5036,13.22,74,100,1.79,NZ,1730082790
566,566,detroit lakes,46.8172,-95.8453,9.03,66,0,4.12,US,1730082792
572,572,ixtapa,20.7000,-105.2000,26.57,69,0,2.06,MX,1730082800
574,574,korwai,24.1167,78.0419,24.54,57,60,2.10,IN,1730082802


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = new_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
10,port shepstone,ZA,-30.7414,30.4550,69,
13,invercargill,NZ,-46.4000,168.3500,71,
18,shanwei,CN,22.7820,115.3475,65,
22,bethel,US,41.3712,-73.4140,56,
41,floresville,US,29.1336,-98.1561,62,
...,...,...,...,...,...,...
564,dunedin,NZ,-45.8742,170.5036,74,
566,detroit lakes,US,46.8172,-95.8453,66,
572,ixtapa,MX,20.7000,-105.2000,69,
574,korwai,IN,24.1167,78.0419,57,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000 
limit = 20
categories = f'accommodation.hotel'
params = {"limit":limit,
          "apiKey":geoapify_key,
          "categories":categories
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,"Lat"]
    longitude = hotel_df.loc[index,"Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port shepstone - nearest hotel: The Spot Backpackers'
invercargill - nearest hotel: The Grand
shanwei - nearest hotel: 汕尾保利希尔顿逸林酒店
bethel - nearest hotel: Hampton Inn Danbury
floresville - nearest hotel: Holiday Inn Express & Suites Floresville
kill devil hills - nearest hotel: Mariner Days Inn & Suites
sindgi - nearest hotel: Sangam Hotel
klyuchi - nearest hotel: No hotel found
praia da vitoria - nearest hotel: Salles
nurota - nearest hotel: Nurota
myitkyina - nearest hotel: New Light Hotel
vernon - nearest hotel: Okanagan Royal Park Inn
albany - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
salto - nearest hotel: Hotel Español Salto
saint-pierre - nearest hotel: Tropic Hotel
mercedes - nearest hotel: Gran Hotel Mercedes
mutsu - nearest hotel: とびない旅館　本館
el salto - nearest hotel: No hotel found
erenhot - nearest hotel: No hotel found
mairang - nearest hotel: No hotel found
san nicolas de los arroyos - nearest hotel: Citex
hanzhong - ne

,City,Country,Lat,Lng,Humidity,Hotel Name
10,port shepstone,ZA,-30.7414,30.4550,69,The Spot Backpackers'
13,invercargill,NZ,-46.4000,168.3500,71,The Grand
18,shanwei,CN,22.7820,115.3475,65,汕尾保利希尔顿逸林酒店
22,bethel,US,41.3712,-73.4140,56,Hampton Inn Danbury
41,floresville,US,29.1336,-98.1561,62,Holiday Inn Express & Suites Floresville
...,...,...,...,...,...,...
564,dunedin,NZ,-45.8742,170.5036,74,Scenic Hotel Dunedin City
566,detroit lakes,US,46.8172,-95.8453,66,Graystone Hotel
572,ixtapa,MX,20.7000,-105.2000,69,Ma. Cristina
574,korwai,IN,24.1167,78.0419,57,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    alpha = 0.4,
    scale = 0.75,
    color = "City",
    hover_cols = ["Country","Hotel Name"])

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)